In [12]:
!pip install pandas numpy tqdm datasets sentence-transformers faiss-cpu langchain openai streamlit langchain_community
!pip install transformers[torch]
!pip install accelerate>=0.26.0
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu128

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


  Using cached torch-2.6.0-cp310-cp310-manylinux1_x86_64.whl.metadata (28 kB)
  Using cached nvidia_cuda_nvrtc_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-9.1.0.70-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.4.5.8-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.2.1.3-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.5.147-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.6.1.9-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.3.1.170-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparselt_cu12-0.6.2-py3

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Looking in indexes: https://download.pytorch.org/whl/cu128
  Using cached https://download.pytorch.org/whl/cu128/torch-2.7.1%2Bcu128-cp310-cp310-manylinux_2_28_x86_64.whl.metadata (29 kB)
  Using cached https://download.pytorch.org/whl/sympy-1.13.3-py3-none-any.whl.metadata (12 kB)
  Using cached https://download.pytorch.org/whl/cu128/nvidia_cuda_nvrtc_cu12-12.8.61-py3-none-manylinux2010_x86_64.manylinux_2_12_x86_64.whl.metadata (1.7 kB)
  Using cached https://download.pytorch.org/whl/cu128/nvidia_cuda_runtime_cu12-12.8.57-py3-none-manylinux2014_x86_64.manylinux_2_17_x86_64.whl.metadata (1.7 kB)
  Using cached https://download.pytorch.org/whl/cu128/nvidia_cuda_cupti_cu12-12.8.57-py3-none-manylinux2014_x86_64.manylinux_2_17_x86_64.whl.metadata (1.7 kB)
  Using cached https://download.pytorch.org/whl/cu128/nvidia_cudnn_cu12-9.7.1.26-py3-none-manylinux_2_27_x86_64.whl.metadata (1.8 kB)
  Using cached https://download.pytorch.org/whl/cu128/nvidia_cublas_cu12-12.8.3.14-py3-none-manylinux_2_

In [ ]:

import os
import json
import pandas as pd
import numpy as np
import re
from tqdm.auto import tqdm

from datasets import load_dataset

from transformers import AutoTokenizer, AutoModel
from sentence_transformers import SentenceTransformer, InputExample, losses

import faiss

from langchain import OpenAI, PromptTemplate
from langchain.chains import RetrievalQA
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings

import streamlit as st

import torch

from sentence_transformers import SentenceTransformer
import faiss
import pandas as pd
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.chat_models import ChatOpenAI
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain.docstore.document import Document
from dotenv import load_dotenv
import os

import json
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, pipeline
from langchain import HuggingFacePipeline
from langchain.chains import RetrievalQA
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS



# Load environment variables from .env file
load_dotenv()

True

In [2]:
# 1. Data Preparation
ds = load_dataset("Abirate/english_quotes")
df = pd.DataFrame(ds["train"])
df = df.dropna(subset=["quote","author"])
df["quote_clean"] = df["quote"].str.strip()
df.to_csv("english_quotes_clean.csv", index=False)


In [3]:
# 2. Model Fine‑Tuning
train_examples = []
for row in tqdm(df.sample(frac=0.2, random_state=42).itertuples(), total=int(len(df)*0.2)):
    text = row.quote_clean
    label = f"{row.author} | {','.join(row.tags)}"
    train_examples.append(InputExample(texts=[text, label]))
model = SentenceTransformer('all-MiniLM-L6-v2')
train_dataloader = torch.utils.data.DataLoader(train_examples, shuffle=True, batch_size=32)
train_loss = losses.MultipleNegativesRankingLoss(model)
model.fit(train_objectives=[(train_dataloader, train_loss)], epochs=2, warmup_steps=100, show_progress_bar=True)
model.save("fine_tuned_quote_sbert")

502it [00:00, 307019.63it/s]           


Step,Training Loss


In [ ]:
!pip install python-dotenv


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [13]:
from langchain_openai import AzureChatOpenAI

load_dotenv()

# Load and encode texts
ft_model = SentenceTransformer("fine_tuned_quote_sbert")
texts = df["quote_clean"].dropna().tolist()

# Convert to LangChain Document objects
documents = [Document(page_content=text) for text in texts]

# Define embedding model
hf_embed = HuggingFaceEmbeddings(model_name="fine_tuned_quote_sbert")

# Create FAISS vector store
faiss_store = FAISS.from_documents(documents, embedding=hf_embed)

# ✅ Save the vector store for later loading
faiss_store.save_local("faiss_index")

# LLM
# llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)

AZURE_API_KEY     = os.getenv("AZURE_OPENAI_API_KEY")
AZURE_ENDPOINT    = os.getenv("AI_BASE")
AZURE_DEPLOYMENT  = os.getenv("AZURE_DEPLOYMENT")
AZURE_API_VERSION = os.getenv("AZURE_OPENAI_VERSION")


llm = AzureChatOpenAI(
    api_key=AZURE_API_KEY,
    azure_endpoint=AZURE_ENDPOINT,
    deployment_name=AZURE_DEPLOYMENT,
    api_version=AZURE_API_VERSION,
    temperature=0,
    # max_tokens=2000,
)

# Prompt
prompt = PromptTemplate(
    template="""
You are a quote assistant. Use the following retrieved quotes to answer the user query.
{context}

User Query: {question}

Provide: 
- JSON with fields: quotes (list), authors, tags, similarity_scores
""",
    input_variables=["context", "question"]
)

# RAG QA Chain
qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=faiss_store.as_retriever(search_kwargs={"k": 5}),
    return_source_documents=True,
    chain_type_kwargs={"prompt": prompt}
)


In [14]:


# …later, when you load the index:
hf_embed = HuggingFaceEmbeddings(model_name="fine_tuned_quote_sbert")

# This ensures the pickle file inside your FAISS folder can be deserialized:
db = FAISS.load_local(
    "faiss_index", 
    hf_embed, 
    allow_dangerous_deserialization=True
)

retriever = db.as_retriever()


# === 1. Load Local Model ===
model_name = "google/flan-t5-small"  # Change to flan-t5-base or another if needed

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

llm_pipeline = pipeline("text2text-generation", model=model, tokenizer=tokenizer, max_length=256)
llm = HuggingFacePipeline(pipeline=llm_pipeline)

# === 2. Use Existing Retriever ===

# === 3. Local RAG Evaluation ===
test_queries = [
    "Quotes about insanity attributed to Einstein",
    "Motivational quotes tagged 'accomplishment'",
    "All Oscar Wilde quotes with humor"
]

for q in test_queries:
    res = qa(q)

    # Reduce to top-1 source for fewer tokens
    sources = res.get("source_documents", [])[:1]

    result_json = {
        "query": q,
        "answer": res["result"],
        "sources": [
            {
                "quote": doc.page_content,
                "author": doc.metadata.get("author"),
                "tags": doc.metadata.get("tags"),
                "score": doc.metadata.get("score")
            }
            for doc in sources
        ]
    }

    print(json.dumps(result_json, indent=2))


Device set to use cuda:0


{
  "query": "Quotes about insanity attributed to Einstein",
  "answer": "```json\n{\n  \"quotes\": [\n    \"In individuals, insanity is rare; but in groups, parties, nations and epochs, it is the rule.\",\n    \"No great mind has ever existed without a touch of madness.\",\n    \"There's a fine line between genius and insanity. I have erased this line.\",\n    \"I don't suffer from my insanity -- I enjoy every minute of it.\",\n    \"I became insane, with long intervals of horrible sanity.\"\n  ],\n  \"authors\": [\n    \"Unknown\",\n    \"Aristotle\",\n    \"Unknown\",\n    \"Unknown\",\n    \"Edgar Allan Poe\"\n  ],\n  \"tags\": [\n    \"insanity\",\n    \"genius\",\n    \"madness\",\n    \"mental health\"\n  ],\n  \"similarity_scores\": [\n    0.8,\n    0.7,\n    0.9,\n    0.85,\n    0.75\n  ]\n}\n```",
  "sources": [
    {
      "quote": "\u201cIn individuals, insanity is rare; but in groups, parties, nations and epochs, it is the rule.\u201d",
      "author": null,
      "tags": 

In [17]:
# 5. Streamlit App
st.title("Semantic Quote Retrieval with RAG")
query = st.text_input("Enter your query:", "")
if query:
    result = qa(query)
    st.subheader("JSON Response")
    st.json({
        "response": result["result"],
        "sources": [
            {"quote": doc.page_content, "author": doc.metadata.get("author"), "tags": doc.metadata.get("tags"), "score": doc.metadata.get("score")}
            for doc in result["source_documents"]
        ]
    })
    st.subheader("Top Retrieved Quotes")
    for doc in result["source_documents"]:
        st.write(f"> {doc.page_content}")
        st.caption(f"Author: {doc.metadata.get('author')} – Score: {doc.metadata.get('score'):.3f}")

2025-06-12 09:50:11.465 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-12 09:50:11.466 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-12 09:50:11.467 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-12 09:50:11.468 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-12 09:50:11.469 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-12 09:50:11.470 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-12 09:50:11.471 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-12 09:50:11.472 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar